In [2]:
import torch

# 加载 .pt 文件
hidden_states = torch.load('/mnt/ceph/develop/jiawei/model_checkpoint/ckpts_2b_lora/lora-disney-09-08-20-59/video/latents/latents_gs-000900/latent.pt')

hidden_states.shape
# torch.Size([2, 4, 16, 60, 90])

torch.Size([1, 16, 13, 60, 90])

In [5]:

from diffusers.image_processor import VaeImageProcessor


def _vae3d_decode(vae3d, latents, denormalize=True):
      
    latents = latents.to(vae3d.dtype)
    latents = latents.permute(0, 2, 1, 3, 4)  # to Shape: [B, T_num_channels, chunk_size, H, W]
    latents = 1 / vae3d.config.scaling_factor * latents

    frames = vae3d.decode(latents).sample # B, C, F, H, W

    batch_size = frames.shape[0]
    outputs = []
    for batch_idx in range(batch_size):
        pt_image = frames[batch_idx].permute(1, 0, 2, 3)  # (to [num_frames, num_channels, w, h])

        outputs.append(torch.stack(
            [VaeImageProcessor.denormalize(pt_image[i]) if denormalize else  pt_image[i] for i in range(pt_image.shape[0]) ]
        ))
 
    return torch.stack(outputs)

/mnt/ceph/develop/jiawei/conda_env/keyframe_extra/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from diffusers.models import AutoencoderKLCogVideoX

vae = AutoencoderKLCogVideoX.from_pretrained('/mnt/ceph/develop/jiawei/model_checkpoint/CogVideoX-2b', subfolder="vae")

vae.to(torch.bfloat16)

The config attributes {'mid_block_add_attention': True, 'sample_size': 256} were passed to AutoencoderKLCogVideoX, but are not expected and will be ignored. Please verify your config.json configuration file.


AutoencoderKLCogVideoX(
  (encoder): CogVideoXEncoder3D(
    (conv_in): CogVideoXCausalConv3d(
      (conv): CogVideoXSafeConv3d(3, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    )
    (down_blocks): ModuleList(
      (0): CogVideoXDownBlock3D(
        (resnets): ModuleList(
          (0-2): 3 x CogVideoXResnetBlock3D(
            (nonlinearity): SiLU()
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): CogVideoXCausalConv3d(
              (conv): CogVideoXSafeConv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
            )
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): CogVideoXCausalConv3d(
              (conv): CogVideoXSafeConv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
            )
          )
        )
        (downsamplers): ModuleList(
          (0): CogVideoXDownsample3D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), strid

In [9]:
images = _vae3d_decode(vae,hidden_states )


RuntimeError: Given groups=1, weight of size [512, 16, 3, 3, 3], expected input[1, 13, 4, 62, 92] to have 16 channels, but got 13 channels instead

In [4]:



import sys
sys.path.append('/mnt/ceph/develop/jiawei/CogVideo')

In [5]:


import os
import math
import argparse
from typing import List, Union
from tqdm import tqdm
from omegaconf import ListConfig
import imageio

import torch
import numpy as np
from einops import rearrange
import torchvision.transforms as TT

from sat.model.base_model import get_model
from sat.training.model_io import load_checkpoint
from sat import mpu
 
from arguments import get_args 
 
py_parser = argparse.ArgumentParser(add_help=False)
known, args_list = py_parser.parse_known_args(args=[

                "--base",
                "/mnt/ceph/develop/jiawei/CogVideo/sat/configs/cogvideox_2b.yaml",
                "/mnt/ceph/develop/jiawei/CogVideo/sat/configs/inference.yaml",
                "--seed",
                "42"
    
])

 

ModuleNotFoundError: No module named 'sat.model'

In [ ]:

args = get_args(args_list)
args = argparse.Namespace(**vars(args), **vars(known))
del args.deepspeed_config
args.model_config.first_stage_config.params.cp_size = 1
args.model_config.network_config.params.transformer_args.model_parallel_size = 1
args.model_config.network_config.params.transformer_args.checkpoint_activations = False
args.model_config.loss_fn_config.params.sigma_sampler_config.params.uniform_sampling = False
model = instantiate_from_config(config).eval()
model.train = disabled_train
for param in model.parameters():
    param.requires_grad = False